<a href="https://colab.research.google.com/github/luisgallegosc/parallel-algorithms/blob/main/FormaParalela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <fstream>
#include <sstream>

//#=============================================================================
//#=========================   FORMA PARALELA   ================================
//#=============================================================================
//#LA PARALELIZACION ES EN ESCALA DE GRISES
//#CONTIENE EL CONTADOR DE TIEMPO

using namespace std;
using std::cout; using std::cerr;
using std::endl; using std::string;
using std::ifstream;

#define MAX_DIGITS 10
// declaracion de funciones
// defines
__device__ int Multiplo(int a, int b){
    if(a%b==0){return 1;}
    else{return 0;}
}
// #GLOBAL : funcion llamada desde el host y ejecutada en el device ( blur  kernel )
__global__ void blur( int *A , int *B , int filass,int columnass) {
// # indice de columna Thread
int columnaThread = threadIdx.y ;
// # indice de fila Thread
int filaThread = threadIdx.x ;
// # indice de columna Block
int columnaBlock = blockIdx.y ;
// # indice de fila Block
int filaBlock = blockIdx.x ;

int x=filaThread+filaBlock*blockDim.x;
int y=columnaThread+columnaBlock*blockDim.y;

int myID = y + (x * gridDim.y*gridDim.x);
if((myID>0 && myID<columnass) || (Multiplo(myID,columnass)==1) || ((Multiplo(myID+1,columnass)==1)) || (myID>(filass-1)*(columnass)-1 && myID<filass*columnass) ){
    B[myID] = 255;
}
else{
    // # Termino 1
    int T1=0.0625*A[myID-columnass-1];
    // # Termino 2
    int T2=0.125*A[myID-columnass];
    // # Termino 3
    int T3=0.0625*A[myID-columnass+1];
    // # Termino 4
    int T4=0.125*A[myID-1];
    // # Termino 5
    int T5=0.25*A[myID];
    // # Termino 6
    int T6=0.125*A[myID+1];
    // # Termino 7
    int T7=0.0625*A[myID+columnass-1];
    // # Termino 8
    int T8=0.125*A[myID+columnass];
    // # Termino 9
    int T9=0.0625*A[myID+columnass+1];
    B[myID] = T1+T2+T3+T4+T5+T6+T7+T8+T9;
}
}

// #GLOBAL : funcion llamada desde el host y ejecutada en el device ( bottom sobel  kernel )
__global__ void bottomsobel( int *A , int *B , int filass,int columnass) {
// # indice de columna Thread
int columnaThread = threadIdx.y ;
// # indice de fila Thread
int filaThread = threadIdx.x ;
// # indice de columna Block
int columnaBlock = blockIdx.y ;
// # indice de fila Block
int filaBlock = blockIdx.x ;

int x=filaThread+filaBlock*blockDim.x;
int y=columnaThread+columnaBlock*blockDim.y;

int myID = y + (x * blockDim.x*gridDim.x);
if((myID>0 && myID<columnass) || (Multiplo(myID,columnass)==1) || ((Multiplo(myID+1,columnass)==1)) || (myID>(filass-1)*(columnass)-1 && myID<filass*columnass) ){
    B[myID] = 255;
}
else{
    // # Termino 1
     int T1=-1*A[myID-columnass-1];
    // # Termino 2
     int T2=-2*A[myID-columnass];
    // # Termino 3
     int T3=-1*A[myID-columnass+1];
    // # Termino 4
     int T4=0*A[myID-1];
    // # Termino 5
     int T5=0*A[myID];
    // # Termino 6
     int T6=0*A[myID+1];
    // # Termino 7
     int T7=1*A[myID+columnass-1];
    // # Termino 8
     int T8=2*A[myID+columnass];
    // # Termino 9
     int T9=1*A[myID+columnass+1];
    B[myID] = (T1+T2+T3+T4+T5+T6+T7+T8+T9)*1.0;
}
}

// #GLOBAL : funcion llamada desde el host y ejecutada en el device ( emboss   kernel )
__global__ void emboss( int *A , int *B , int filass,int columnass) {
// # indice de columna Thread
int columnaThread = threadIdx.y ;
// # indice de fila Thread
int filaThread = threadIdx.x ;
// # indice de columna Block
int columnaBlock = blockIdx.y ;
// # indice de fila Block
int filaBlock = blockIdx.x ;

int x=filaThread+filaBlock*blockDim.x;
int y=columnaThread+columnaBlock*blockDim.y;

int myID = y + (x * blockDim.x*gridDim.x);
if((myID>0 && myID<columnass) || (Multiplo(myID,columnass)==1) || ((Multiplo(myID+1,columnass)==1)) || (myID>(filass-1)*(columnass)-1 && myID<filass*columnass) ){
    B[myID] = 255;
}
else{
    // # Termino 1
     int T1=-2*A[myID-columnass-1];
    // # Termino 2
     int T2=-1*A[myID-columnass];
    // # Termino 3
     int T3=0*A[myID-columnass+1];
    // # Termino 4
     int T4=-1*A[myID-1];
    // # Termino 5
     int T5=1*A[myID];
    // # Termino 6
     int T6=1*A[myID+1];
    // # Termino 7
     int T7=0*A[myID+columnass-1];
    // # Termino 8
     int T8=1*A[myID+columnass];
    // # Termino 9
     int T9=2*A[myID+columnass+1];
    B[myID] = (T1+T2+T3+T4+T5+T6+T7+T8+T9)*1.0;
}
}
// #GLOBAL : funcion llamada desde el host y ejecutada en el device ( identity   kernel )
__global__ void identity( int *A , int *B , int filass,int columnass) {
// # indice de columna Thread
int columnaThread = threadIdx.y ;
// # indice de fila Thread
int filaThread = threadIdx.x ;
// # indice de columna Block
int columnaBlock = blockIdx.y ;
// # indice de fila Block
int filaBlock = blockIdx.x ;

int x=filaThread+filaBlock*blockDim.x;
int y=columnaThread+columnaBlock*blockDim.y;

int myID = y + (x * blockDim.x*gridDim.x);
if((myID>0 && myID<columnass) || (Multiplo(myID,columnass)==1) || ((Multiplo(myID+1,columnass)==1)) || (myID>(filass-1)*(columnass)-1 && myID<filass*columnass) ){
    B[myID] = 255;
}
else{
    // # Termino 1
     int T1=0*A[myID-columnass-1];
    // # Termino 2
     int T2=0*A[myID-columnass];
    // # Termino 3
     int T3=0*A[myID-columnass+1];
    // # Termino 4
     int T4=0*A[myID-1];
    // # Termino 5
     int T5=1*A[myID];
    // # Termino 6
     int T6=0*A[myID+1];
    // # Termino 7
     int T7=0*A[myID+columnass-1];
    // # Termino 8
     int T8=0*A[myID+columnass];
    // # Termino 9
     int T9=0*A[myID+columnass+1];
    B[myID] = T1+T2+T3+T4+T5+T6+T7+T8+T9;
}
}

// #GLOBAL : funcion llamada desde el host y ejecutada en el device (left sobel  kernel )
__global__ void leftsobel( int *A , int *B , int filass,int columnass) {
// # indice de columna Thread
int columnaThread = threadIdx.y ;
// # indice de fila Thread
int filaThread = threadIdx.x ;
// # indice de columna Block
int columnaBlock = blockIdx.y ;
// # indice de fila Block
int filaBlock = blockIdx.x ;

int x=filaThread+filaBlock*blockDim.x;
int y=columnaThread+columnaBlock*blockDim.y;

int myID = y + (x * blockDim.x*gridDim.x);
if((myID>0 && myID<columnass) || (Multiplo(myID,columnass)==1) || ((Multiplo(myID+1,columnass)==1)) || (myID>(filass-1)*(columnass)-1 && myID<filass*columnass) ){
    B[myID] = 255;
}
else{
    // # Termino 1
     int T1=1*A[myID-columnass-1];
    // # Termino 2
     int T2=0*A[myID-columnass];
    // # Termino 3
     int T3=-1*A[myID-columnass+1];
    // # Termino 4
     int T4=2*A[myID-1];
    // # Termino 5
     int T5=0*A[myID];
    // # Termino 6
     int T6=-2*A[myID+1];
    // # Termino 7
     int T7=1*A[myID+columnass-1];
    // # Termino 8
     int T8=0*A[myID+columnass];
    // # Termino 9
     int T9=-1*A[myID+columnass+1];
    B[myID] = T1+T2+T3+T4+T5+T6+T7+T8+T9;
}
}

// #GLOBAL : funcion llamada desde el host y ejecutada en el device ( outline kernel )
__global__ void outline( int *A , int *B , int filass,int columnass) {
// # indice de columna Thread
int columnaThread = threadIdx.y ;
// # indice de fila Thread
int filaThread = threadIdx.x ;
// # indice de columna Block
int columnaBlock = blockIdx.y ;
// # indice de fila Block
int filaBlock = blockIdx.x ;

int x=filaThread+filaBlock*blockDim.x;
int y=columnaThread+columnaBlock*blockDim.y;

int myID = y + (x * blockDim.x*gridDim.x);
if((myID>0 && myID<columnass) || (Multiplo(myID,columnass)==1) || ((Multiplo(myID+1,columnass)==1)) || (myID>(filass-1)*(columnass)-1 && myID<filass*columnass) ){
    B[myID] = 255;
}
else{
    // # Termino 1
     int T1=-1*A[myID-columnass-1];
    // # Termino 2
     int T2=-1*A[myID-columnass];
    // # Termino 3
     int T3=-1*A[myID-columnass+1];
    // # Termino 4
     int T4=-1*A[myID-1];
    // # Termino 5
     int T5=8*A[myID];
    // # Termino 6
     int T6=-1*A[myID+1];
    // # Termino 7
     int T7=-1*A[myID+columnass-1];
    // # Termino 8
     int T8=-1*A[myID+columnass];
    // # Termino 9
     int T9=-1*A[myID+columnass+1];
    B[myID] = T1+T2+T3+T4+T5+T6+T7+T8+T9;
}
}

// #GLOBAL : funcion llamada desde el host y ejecutada en el device ( right sobel  kernel )
__global__ void rightsobel( int *A , int *B , int filass,int columnass) {
// # indice de columna Thread
int columnaThread = threadIdx.y ;
// # indice de fila Thread
int filaThread = threadIdx.x ;
// # indice de columna Block
int columnaBlock = blockIdx.y ;
// # indice de fila Block
int filaBlock = blockIdx.x ;

int x=filaThread+filaBlock*blockDim.x;
int y=columnaThread+columnaBlock*blockDim.y;

int myID = y + (x * blockDim.x*gridDim.x);
if((myID>0 && myID<columnass) || (Multiplo(myID,columnass)==1) || ((Multiplo(myID+1,columnass)==1)) || (myID>(filass-1)*(columnass)-1 && myID<filass*columnass) ){
    B[myID] = 255;
}
else{
    // # Termino 1
     int T1=-1*A[myID-columnass-1];
    // # Termino 2
     int T2=0*A[myID-columnass];
    // # Termino 3
     int T3=1*A[myID-columnass+1];
    // # Termino 4
     int T4=-2*A[myID-1];
    // # Termino 5
     int T5=0*A[myID];
    // # Termino 6
     int T6=2*A[myID+1];
    // # Termino 7
     int T7=-1*A[myID+columnass-1];
    // # Termino 8
     int T8=0*A[myID+columnass];
    // # Termino 9
     int T9=1*A[myID+columnass+1];
    B[myID] = T1+T2+T3+T4+T5+T6+T7+T8+T9;
}
}

// #GLOBAL : funcion llamada desde el host y ejecutada en el device (sharpen   kernel )
__global__ void sharpen( int *A , int *B , int filass,int columnass) {
// # indice de columna Thread
int columnaThread = threadIdx.y ;
// # indice de fila Thread
int filaThread = threadIdx.x ;
// # indice de columna Block
int columnaBlock = blockIdx.y ;
// # indice de fila Block
int filaBlock = blockIdx.x ;

int x=filaThread+filaBlock*blockDim.x;
int y=columnaThread+columnaBlock*blockDim.y;

int myID = y + (x * blockDim.x*gridDim.x);
if((myID>0 && myID<columnass) || (Multiplo(myID,columnass)==1) || ((Multiplo(myID+1,columnass)==1)) || (myID>(filass-1)*(columnass)-1 && myID<filass*columnass) ){
    B[myID] = 255;
}
else{
    // # Termino 1
     int T1=0*A[myID-columnass-1];
    // # Termino 2
     int T2=-1*A[myID-columnass];
    // # Termino 3
     int T3=0*A[myID-columnass+1];
    // # Termino 4
     int T4=-1*A[myID-1];
    // # Termino 5
     int T5=5*A[myID];
    // # Termino 6
     int T6=-1*A[myID+1];
    // # Termino 7
     int T7=0*A[myID+columnass-1];
    // # Termino 8
     int T8=-1*A[myID+columnass];
    // # Termino 9
     int T9=0*A[myID+columnass+1];
    B[myID] = (T1+T2+T3+T4+T5+T6+T7+T8+T9)*1.0;
}
}

// #GLOBAL : funcion llamada desde el host y ejecutada en el device ( top sobel  kernel )
__global__ void topsobel( int *A , int *B , int filass,int columnass) {
// # indice de columna Thread
int columnaThread = threadIdx.y ;
// # indice de fila Thread
int filaThread = threadIdx.x ;
// # indice de columna Block
int columnaBlock = blockIdx.y ;
// # indice de fila Block
int filaBlock = blockIdx.x ;

int x=filaThread+filaBlock*blockDim.x;
int y=columnaThread+columnaBlock*blockDim.y;

int myID = y + (x * blockDim.x*gridDim.x);
if((myID>0 && myID<columnass) || (Multiplo(myID,columnass)==1) || ((Multiplo(myID+1,columnass)==1)) || (myID>(filass-1)*(columnass)-1 && myID<filass*columnass) ){
    B[myID] = 255;
}
else{
    // # Termino 1
     int T1=1*A[myID-columnass-1];
    // # Termino 2
     int T2=2*A[myID-columnass];
    // # Termino 3
     int T3=1*A[myID-columnass+1];
    // # Termino 4
     int T4=0*A[myID-1];
    // # Termino 5
     int T5=0*A[myID];
    // # Termino 6
     int T6=0*A[myID+1];
    // # Termino 7
     int T7=-1*A[myID+columnass-1];
    // # Termino 8
     int T8=-2*A[myID+columnass];
    // # Termino 9
     int T9=-1*A[myID+columnass+1];
    B[myID] = (T1+T2+T3+T4+T5+T6+T7+T8+T9)*1.0;
}
}


// MAIN : rutina principal ejecutada en el host
int main(int argc , char **argv ) {
  //MATRIZ
  string filename("/content/mario.txt");
  int number;
  int filas;
  int columnas;
  float arreglo[1000000];
  int contador=0;
  ifstream input_file(filename);
  if (!input_file.is_open()) {
      cerr << "Could not open the file - '"
      << filename << "'" << endl;
     return EXIT_FAILURE;
  }
  int i=0;
  while (input_file >> number) {
      if(contador==0){filas=number;contador++;}
      else{
      if(contador==1){columnas=number;contador++;}
      else{
          arreglo[i]=number;
          i++;
      }
    }
  }
  int R[filas * columnas];
  for(int i=0;i<filas * columnas;i++){
      R[i]=arreglo[i];
  }

  // # declaraciones
  int *hst_AR , *hst_BR ,*hst_AG , *hst_BG ,*hst_AB , *hst_BB ;
  int *dev_AR , *dev_BR ,*dev_AG , *dev_BG ,*dev_AB , *dev_BB ;
  // # reserva en el host
  hst_AR = ( int *) malloc ( filas * columnas * sizeof ( int ) ) ;
  hst_BR = ( int *) malloc ( filas * columnas * sizeof ( int ) ) ;
  hst_AG = ( int *) malloc ( filas * columnas * sizeof ( int ) ) ;
  hst_BG = ( int *) malloc ( filas * columnas * sizeof ( int ) ) ;
  hst_AB = ( int *) malloc ( filas * columnas * sizeof ( int ) ) ;
  hst_BB = ( int *) malloc ( filas * columnas * sizeof ( int ) ) ;
  // # reserva en el device
  cudaMalloc ( ( void**) &dev_AR , filas * columnas * sizeof ( int ) ) ;
  cudaMalloc ( ( void**) &dev_BR , filas * columnas * sizeof ( int ) ) ;
  cudaMalloc ( ( void**) &dev_AG , filas * columnas * sizeof ( int ) ) ;
  cudaMalloc ( ( void**) &dev_BG , filas * columnas * sizeof ( int ) ) ;
  cudaMalloc ( ( void**) &dev_AB , filas * columnas * sizeof ( int ) ) ;
  cudaMalloc ( ( void**) &dev_BB , filas * columnas * sizeof ( int ) ) ;
  // # incializacion
  for (int i =0; i < filas*columnas ; i ++) {
      hst_AR[ i ] = R[i];

  }
  // # copia de datos
  cudaMemcpy ( dev_AR , hst_AR , filas * columnas * sizeof ( int ) ,cudaMemcpyHostToDevice ) ;

  // # dimensiones del kernel
  dim3 Nbloques(20,20) ;
  dim3 hilosB(20,20) ;


  // #Medicion del Tiempo
   // declaracion de eventos
   cudaEvent_t start ;
   cudaEvent_t stop ;
   // creacion de eventos
   cudaEventCreate (&start ) ;
   cudaEventCreate (&stop ) ;
   // marca de inicio
   cudaEventRecord ( start ,0) ;

  //# llamada al kernel bidimensional de filasxcolumnas hilos
  emboss<<<Nbloques,hilosB>>>( dev_AR , dev_BR ,filas,columnas) ;

  //#Tiempo
   // marca de final
   cudaEventRecord ( stop ,0) ;
   // sincronizacion GPU -CPU
   cudaEventSynchronize ( stop ) ;
   // calculo del tiempo en milisegundos
   float elapsedTime ;
   cudaEventElapsedTime (&elapsedTime , start , stop ) ;
   // impresion de resultados
   printf ("> Tiempo de ejecucion : %f ms\n", elapsedTime ) ;
   // liberacion de recursos
   cudaEventDestroy ( start ) ;
   cudaEventDestroy ( stop ) ;


  // # recogida de datos
  cudaMemcpy( hst_BR , dev_BR ,filas * columnas * sizeof( int ) ,cudaMemcpyDeviceToHost ) ;

  int ArregloTotal[filas*columnas*3];
  for (int i =0; i < filas*columnas ; i ++){
      ArregloTotal[i]=hst_BR[i];
  }


  ofstream archivo;
  archivo.open("/content/nuevo.txt",ios::out);
  if(archivo.fail()){
      cout<<"No se pudo abri el archivo";
  }


  char cadena_decimal[50];

  snprintf(cadena_decimal, 50, "%i", filas);
  archivo<<cadena_decimal<<" ";
  snprintf(cadena_decimal, 50, "%i", columnas);
  archivo<<cadena_decimal<<"\n";

  for (int i =0; i < filas*columnas ; i ++){
      snprintf(cadena_decimal, 50, "%i", ArregloTotal[i]);
      archivo<<cadena_decimal<<" ";
  }
  archivo<<cadena_decimal<<"\n";
  for (int i =0; i < filas*columnas ; i ++){
      snprintf(cadena_decimal, 50, "%i", ArregloTotal[i]);
      archivo<<cadena_decimal<<" ";
  }
  archivo<<cadena_decimal<<"\n";
  for (int i =0; i < filas*columnas ; i ++){
      snprintf(cadena_decimal, 50, "%i", ArregloTotal[i]);
      archivo<<cadena_decimal<<" ";
  }
  archivo.close();


  // # salida
  printf ("\n SE EJECUTO EXITOSAMENTE") ;
  fflush ( stdin ) ;
  char tecla = getchar () ;
  return EXIT_SUCCESS;
}

> Tiempo de ejecucion : 0.349792 ms

 SE EJECUTO EXITOSAMENTE
